In [1]:
import os
import re
import sys
import pandas as pd
import numpy as np

# CURRENT FEATURES

## FAST_MOVE_DPS
**Description:** This feature calculates the damage per second (DPS) of a Pokémon's fast move. It quantifies the average damage inflicted per second during the fast move's execution.

**Calculation:** FAST_MOVE_DPS = Fast Move Power / Fast Move Duration
Use: This metric is useful for comparing the base effectiveness of different fast moves independently of their type or other effects. It helps in assessing which fast moves are inherently faster at dealing damage, guiding players in selecting Pokémon for their speed in battle.

## CHARGE_MOVE_DPS

**Description:** Similar to FAST_MOVE_DPS, this feature calculates the damage per second for charge moves, which are special attacks that require energy accumulated from fast moves.

**Calculation:** CHARGE_MOVE_DPS = Charge Move Power / Charge Move Duration
Use: Charge moves are decisive in battles, and understanding their DPS helps players optimize when to unleash them for maximum effect, considering the trade-off between damage output and the time taken to execute.

## FAST_MOVE_EPS

**Description:** This feature measures the energy per second (EPS) that a fast move generates. EPS is crucial as it determines how quickly a Pokémon can charge its powerful charge moves.

**Calculation:** FAST_MOVE_EPS = Fast Energy Boost / Fast Move Duration
Use: EPS is a key component in battle strategy, especially in choosing fast moves that balance damage output with energy generation, enabling quicker access to charge moves.

## SYNERGY_SCORE
**Description:** This score evaluates how well a Pokémon's fast move complements its charge move in terms of energy dynamics. A higher synergy score indicates a more efficient energy cycle between the fast and charge moves.

**Calculation:** SYNERGY_SCORE = (Fast Move EPS * Charge Move Energy Cost) / Charge Move Power
Use: It helps in selecting movesets that optimize the Pokémon's ability to perform in battles by quickly charging and effectively using their charge moves.

## TYPE_ALIGNMENT_FAST & TYPE_ALIGNMENT_CHARGE

**Description:** These binary features indicate whether the Pokémon's move type matches one of its own types, affecting its eligibility for Same Type Attack Bonus (STAB).

**Calculation:**
TYPE_ALIGNMENT_FAST = TRUE if FAST_MOVE_TYPE matches either TYPE_ONE or TYPE_TWO
TYPE_ALIGNMENT_CHARGE = TRUE if CHARGED_MOVE_TYPE matches either TYPE_ONE or TYPE_TWO
**Use:** These indicators are crucial for identifying moves that gain a STAB, thereby enhancing their damage output significantly.

## ADJUSTED_FAST_MOVE_DPS & ADJUSTED_CHARGE_MOVE_DPS

**Description:** These features adjust the respective DPS values for fast and charge moves if the move qualifies for a STAB, reflecting the enhanced damage due to type alignment.

**Calculation:**
ADJUSTED_FAST_MOVE_DPS = FAST_MOVE_DPS * (1.2 if TYPE_ALIGNMENT_FAST is TRUE, otherwise 1)
ADJUSTED_CHARGE_MOVE_DPS = CHARGE_MOVE_DPS * (1.2 if TYPE_ALIGNMENT_CHARGE is TRUE, otherwise 1)
Use: These adjusted DPS metrics provide a more realistic assessment of a move's performance by incorporating the bonus from STAB, making them vital for comparing the true battle effectiveness of Pokémon moves.

## FAST_MOVE_EFFECTIVENESS

**Description:** This feature measures the effectiveness of a Pokémon's fast move against typical opponents based on type matchups. In Pokémon battles, the effectiveness of moves is determined by the type of the move and the type(s) of the opponent Pokémon. A move can be normal, not very effective, or super effective based on these type interactions.

**Calculation:** FAST_MOVE_EFFECTIVENESS is calculated by averaging the effectiveness of the fast move against both possible types (TYPE_ONE and TYPE_TWO) of an opponent. The effectiveness values are derived from a predefined type effectiveness chart that indicates how effective a particular type is against another. If a Pokémon's move type (FAST_MOVE_TYPE) is super effective against one of the opponent's types, this will increase the score, while being less effective will lower the score.

**Use:** This metric is crucial for understanding which Pokémon might perform better in certain matchups based solely on their fast move's potential to deal damage effectively. It helps in predicting outcomes and strategizing which Pokémon to select based on their fast move's effectiveness across potential battles.

## CHARGE_MOVE_EFFECTIVENESS

**Description:** Similar to the fast move effectiveness, this feature measures how effective a Pokémon's charge move is against typical opponents. Since charge moves are often more powerful and can turn the tide of battle, understanding their effectiveness is key for strategic play.

**Calculation:** CHARGE_MOVE_EFFECTIVENESS is computed in the same way as fast move effectiveness but uses the charge move’s type (CHARGED_MOVE_TYPE). It averages the effectiveness scores of the charge move against the opponent's possible types.

**Use:** This feature helps to evaluate the strategic impact of a Pokémon’s charge move in battles. Knowing how effective a charge move can be against various opponents allows players to plan their energy accumulation (via fast moves) and move execution more effectively. It’s particularly important for deciding when to use charge moves during battles to maximize damage and for choosing the right Pokémon to counter specific opponents in PvP settings.

## WEATHER_BOOSTED_FAST_DPS & WEATHER_BOOSTED_CHARGE_DPS

**Calculation:**
If the move type matches the weather boost type, multiply the adjusted DPS by 1.2.

**Use:** 
This feature will help simulate the effect of weather on move effectiveness, allowing for dynamic battle predictions that reflect real-time conditions.


## SHADOW_ADJUSTED_FAST_DPS & SHADOW_ADJUSTED_CHARGE_DPS

**Calculation:**
Multiply the adjusted DPS values by 1.2 for shadow Pokémon.

**Use:**
Since shadow Pokémon are frequently used in PvP due to their higher damage output, accurately modeling their enhanced attack capabilities will be crucial for predicting battle outcomes.

In [2]:
data = pd.read_csv('current_dataset_20240421.csv')
data = data.drop(columns='Unnamed: 0')
data['ID'] = data['ID'].astype(str).str.zfill(4)

In [3]:
data.head()

,ID,NAME,TYPE_ONE,TYPE_TWO,FAST_MOVE,FAST_MOVE_POWER,FAST_MOVE_TYPE,FAST_ENERGY_BOOST,FAST_MOVE_DURATION,CHARGE_MOVE,...,FAST_MOVE_DPS,CHARGE_MOVE_DPS,FAST_MOVE_EPS,SYNERGY_SCORE,TYPE_ALIGNMENT_FAST,TYPE_ALIGNMENT_CHARGE,ADJUSTED_FAST_MOVE_DPS,ADJUSTED_CHARGE_MOVE_DPS,FAST_MOVE_EFFECTIVENESS,CHARGE_MOVE_EFFECTIVENESS
0,0001,Bulbasaur,Grass,Poison,Tackle,5.0,Normal,5.0,0.5,Return,...,10.000000,50.000000,10.0,9.428571,False,False,10.0,50.000000,1.0,1.00
1,0001,Bulbasaur,Grass,Poison,Tackle,5.0,Normal,5.0,0.5,Return,...,10.000000,50.000000,10.0,9.428571,False,False,10.0,50.000000,1.0,1.00
2,0001,Bulbasaur,Grass,Poison,Vine Whip,7.0,Grass,6.0,0.6,Power Whip,...,11.666667,34.615385,10.0,5.555556,True,True,14.0,41.538462,0.5,0.50
3,0001,Bulbasaur,Grass,Poison,Vine Whip,7.0,Grass,6.0,0.6,Power Whip,...,11.666667,34.615385,10.0,5.555556,True,True,14.0,41.538462,0.5,0.50
4,0001,Bulbasaur,Grass,Poison,Vine Whip,7.0,Grass,6.0,0.6,Sludge Bomb,...,11.666667,34.782609,10.0,6.250000,True,True,14.0,41.739130,0.5,1.25


In [4]:
# Mapping of types to their favorable weather
weather_boosts = {
    'Sunny': ['Grass', 'Ground', 'Fire'],
    'Rainy': ['Water', 'Electric', 'Bug'],
    'Partly Cloudy': ['Normal', 'Rock'],
    'Cloudy': ['Fairy', 'Fighting', 'Poison'],
    'Windy': ['Dragon', 'Flying', 'Psychic'],
    'Snow': ['Ice', 'Steel'],
    'Fog': ['Dark', 'Ghost']
}

# Function to calculate the weather boost
def calculate_weather_boost(row, weather):
    fast_type_boost = 1.2 if row['FAST_MOVE_TYPE'] in weather_boosts.get(weather, []) else 1.0
    charge_type_boost = 1.2 if row['CHARGED_MOVE_TYPE'] in weather_boosts.get(weather, []) else 1.0
    return fast_type_boost, charge_type_boost

# Assuming you have a column 'CURRENT_WEATHER' in your DataFrame that specifies the current weather
# You can replace 'CURRENT_WEATHER' with your actual weather condition variable or parameter.
if 'CURRENT_WEATHER' in data.columns:
    data[['FAST_WEATHER_BOOST', 'CHARGE_WEATHER_BOOST']] = data.apply(lambda row: calculate_weather_boost(row, row['CURRENT_WEATHER']), axis=1, result_type='expand')
else:
    # If you don't have weather conditions per row, you can set a general weather condition here
    general_weather = 'Sunny'  # Example: change to your current or desired weather condition
    data[['FAST_WEATHER_BOOST', 'CHARGE_WEATHER_BOOST']] = data.apply(lambda row: calculate_weather_boost(row, general_weather), axis=1, result_type='expand')

# Applying the weather boost to the DPS calculations
data['ADJUSTED_FAST_MOVE_DPS_WEATHER'] = data['ADJUSTED_FAST_MOVE_DPS'] * data['FAST_WEATHER_BOOST']
data['ADJUSTED_CHARGE_MOVE_DPS_WEATHER'] = data['ADJUSTED_CHARGE_MOVE_DPS'] * data['CHARGE_WEATHER_BOOST']

In [5]:
data.head()

,ID,NAME,TYPE_ONE,TYPE_TWO,FAST_MOVE,FAST_MOVE_POWER,FAST_MOVE_TYPE,FAST_ENERGY_BOOST,FAST_MOVE_DURATION,CHARGE_MOVE,...,TYPE_ALIGNMENT_FAST,TYPE_ALIGNMENT_CHARGE,ADJUSTED_FAST_MOVE_DPS,ADJUSTED_CHARGE_MOVE_DPS,FAST_MOVE_EFFECTIVENESS,CHARGE_MOVE_EFFECTIVENESS,FAST_WEATHER_BOOST,CHARGE_WEATHER_BOOST,ADJUSTED_FAST_MOVE_DPS_WEATHER,ADJUSTED_CHARGE_MOVE_DPS_WEATHER
0,0001,Bulbasaur,Grass,Poison,Tackle,5.0,Normal,5.0,0.5,Return,...,False,False,10.0,50.000000,1.0,1.00,1.0,1.0,10.0,50.000000
1,0001,Bulbasaur,Grass,Poison,Tackle,5.0,Normal,5.0,0.5,Return,...,False,False,10.0,50.000000,1.0,1.00,1.0,1.0,10.0,50.000000
2,0001,Bulbasaur,Grass,Poison,Vine Whip,7.0,Grass,6.0,0.6,Power Whip,...,True,True,14.0,41.538462,0.5,0.50,1.2,1.2,16.8,49.846154
3,0001,Bulbasaur,Grass,Poison,Vine Whip,7.0,Grass,6.0,0.6,Power Whip,...,True,True,14.0,41.538462,0.5,0.50,1.2,1.2,16.8,49.846154
4,0001,Bulbasaur,Grass,Poison,Vine Whip,7.0,Grass,6.0,0.6,Sludge Bomb,...,True,True,14.0,41.739130,0.5,1.25,1.2,1.0,16.8,41.739130


In [6]:
# Check if a Pokémon is a Shadow Pokémon and apply a 20% bonus to the move DPS and adjust energy costs
def apply_shadow_bonus(row):
    if 'Shadow' in row['NAME']:
        # Increase DPS by 20%
        row['ADJUSTED_FAST_MOVE_DPS_SHADOW'] = row['ADJUSTED_FAST_MOVE_DPS'] * 1.2
        row['ADJUSTED_CHARGE_MOVE_DPS_SHADOW'] = row['ADJUSTED_CHARGE_MOVE_DPS'] * 1.2
        # Increase energy cost for charge moves by 20%
        row['ADJUSTED_CHARGE_MOVE_ENERGY_COST'] = row['CHARGE_MOVE_ENERGY_COST'] * 1.2
    else:
        row['ADJUSTED_FAST_MOVE_DPS_SHADOW'] = row['ADJUSTED_FAST_MOVE_DPS']
        row['ADJUSTED_CHARGE_MOVE_DPS_SHADOW'] = row['ADJUSTED_CHARGE_MOVE_DPS']
        row['ADJUSTED_CHARGE_MOVE_ENERGY_COST'] = row['CHARGE_MOVE_ENERGY_COST']
    return row

# Apply the shadow bonus adjustments
data = data.apply(apply_shadow_bonus, axis=1)

In [7]:
data[15:20]

,ID,NAME,TYPE_ONE,TYPE_TWO,FAST_MOVE,FAST_MOVE_POWER,FAST_MOVE_TYPE,FAST_ENERGY_BOOST,FAST_MOVE_DURATION,CHARGE_MOVE,...,ADJUSTED_CHARGE_MOVE_DPS,FAST_MOVE_EFFECTIVENESS,CHARGE_MOVE_EFFECTIVENESS,FAST_WEATHER_BOOST,CHARGE_WEATHER_BOOST,ADJUSTED_FAST_MOVE_DPS_WEATHER,ADJUSTED_CHARGE_MOVE_DPS_WEATHER,ADJUSTED_FAST_MOVE_DPS_SHADOW,ADJUSTED_CHARGE_MOVE_DPS_SHADOW,ADJUSTED_CHARGE_MOVE_ENERGY_COST
15,0001,Shadow Bulbasaur,Grass,Poison,Vine Whip,7.0,Grass,6.0,0.6,Seed Bomb,...,31.428571,0.5,0.5,1.2,1.2,16.8,37.714286,16.8,37.714286,39.6
16,0001,Bulbasaur,Grass,Poison,Tackle,5.0,Normal,5.0,0.5,Seed Bomb,...,31.428571,1.0,0.5,1.0,1.2,10.0,37.714286,10.0,31.428571,33.0
17,0001,Bulbasaur,Grass,Poison,Tackle,5.0,Normal,5.0,0.5,Seed Bomb,...,31.428571,1.0,0.5,1.0,1.2,10.0,37.714286,10.0,31.428571,33.0
18,0001,Shadow Bulbasaur,Grass,Poison,Vine Whip,7.0,Grass,6.0,0.6,Frustration,...,5.000000,0.5,1.0,1.2,1.0,16.8,5.000000,16.8,6.000000,39.6
19,0001,Shadow Bulbasaur,Grass,Poison,Vine Whip,7.0,Grass,6.0,0.6,Frustration,...,5.000000,0.5,1.0,1.2,1.0,16.8,5.000000,16.8,6.000000,39.6


## ADD LEVELS 

In [8]:
cpm_levels = pd.read_csv('../../datasets/levels.csv')

In [9]:
cpm_levels

,LEVEL,CPM
0,1.0,0.094000
1,1.5,0.135137
2,2.0,0.166398
3,2.5,0.192651
4,3.0,0.215732
...,...,...
95,48.5,0.832803
96,49.0,0.835300
97,49.5,0.837803
98,50.0,0.840300


In [10]:
import math

# Function to calculate maximum CP
def calculate_max_cp(row, ivs=(15, 15, 15), cpm=1):
    attack, defense, hp = ivs
    max_cp = ((row['BASE_ATTACK'] + attack) *
              np.sqrt(row['BASE_DEFENSE'] + defense) *
              np.sqrt(row['BASE_HP'] + hp) * cpm**2) / 10
    return math.floor(max_cp)  # Use floor to round down to the nearest integer

# Determine the maximum CPM for levels below 50 and at level 51
max_cpm_50 = cpm_levels[cpm_levels['LEVEL'] <= 50].iloc[-1]['CPM']
max_cpm_51 = cpm_levels[cpm_levels['LEVEL'] == 51].iloc[-1]['CPM']

# Calculate max CP at level 50 and level 51
data['MAX_CP_50'] = data.apply(lambda row: calculate_max_cp(row, cpm=max_cpm_50), axis=1)
data['MAX_CP_51'] = data.apply(lambda row: calculate_max_cp(row, cpm=max_cpm_51), axis=1)

# Define league caps
great_cap = 1500
ultra_cap = 2500

# Filter Pokémon into leagues based on max CP with and without buddy boost
data['LEAGUE_GREAT_50'] = data['MAX_CP_50'] <= great_cap
data['LEAGUE_ULTRA_50'] = data['MAX_CP_50'] <= ultra_cap
data['LEAGUE_MASTER_50'] = data['MAX_CP_50'] > ultra_cap

data['LEAGUE_GREAT_51'] = data['MAX_CP_51'] <= great_cap
data['LEAGUE_ULTRA_51'] = data['MAX_CP_51'] <= ultra_cap
data['LEAGUE_MASTER_51'] = data['MAX_CP_51'] > ultra_cap

In [11]:
# Display unique values for base stats for Bulbasaur
bulbasaur_stats = data[data['NAME'] == 'Palkia Origin Forme'][['BASE_ATTACK', 'BASE_DEFENSE', 'BASE_HP']].drop_duplicates()
print(bulbasaur_stats)

       BASE_ATTACK  BASE_DEFENSE  BASE_HP
18824        280.0         215.0    189.0
18825        286.0         223.0    189.0


In [13]:
!pip install tqdm

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm

# Assuming 'data' is your DataFrame

# Helper function to check if stats look like they've been modified by Shadow multipliers
def check_shadow_like(row, df):
    shadow_attack = row['BASE_ATTACK'] / 1.2
    shadow_defense = row['BASE_DEFENSE'] / 0.8
    # Check for a close match to any unmodified stat within the dataset
    for _, data in tqdm(df.iterrows(), total=df.shape[0], desc="Checking rows"):
        if np.isclose(data['BASE_ATTACK'], shadow_attack, rtol=1e-05, atol=1e-05) and \
           np.isclose(data['BASE_DEFENSE'], shadow_defense, rtol=1e-05, atol=1e-05):
            return True
    return False

# Apply the check with a progress bar
data['IS_SHADOW_LIKE'] = [check_shadow_like(row, data) for index, row in tqdm(data.iterrows(), total=data.shape[0], desc="Applying shadow check")]

# Correct stats where necessary
def correct_stats(row):
    if row['IS_SHADOW_LIKE'] and 'Shadow' not in row['NAME']:
        # Correct stats assumed to be mistakenly modified
        return row['BASE_ATTACK'] / 1.2, row['BASE_DEFENSE'] / 0.8
    return row['BASE_ATTACK'], row['BASE_DEFENSE']

# Applying corrections with progress bar
data[['BASE_ATTACK', 'BASE_DEFENSE']] = pd.DataFrame(data.apply(correct_stats, axis=1, result_type='expand').tolist(), index=data.index)

# Remove the temporary column
data.drop('IS_SHADOW_LIKE', axis=1, inplace=True)

# Ensure Shadow Pokémon have the right modifications applied
def apply_shadow_modifiers(row):
    if 'Shadow' in row['NAME']:
        return row['BASE_ATTACK'] * 1.2, row['BASE_DEFENSE'] * 0.8
    return row['BASE_ATTACK'], row['BASE_DEFENSE']

# Applying shadow modifiers with progress bar
data[['BASE_ATTACK', 'BASE_DEFENSE']] = pd.DataFrame(data.apply(apply_shadow_modifiers, axis=1, result_type='expand').tolist(), index=data.index)


Checking rows:   0%|                                                               | 0/24200 [00:00<?, ?it/s]

Checking rows:   0%|                                                               | 0/24200 [00:00<?, ?it/s]

Checking rows:   0%|                                                               | 0/24200 [00:00<?, ?it/s]

Checking rows:   0%|                                                               | 0/24200 [00:00<?, ?it/s]

Checking rows:   0%|                                                               | 0/24200 [00:00<?, ?it/s]

Checking rows:   0%|                                                               | 0/24200 [00:00<?, ?it/s]

Checking rows:   0%|                                                               | 0/24200 [00:00<?, ?it/s]

Checking rows:   0%|                                                               | 0/24200 [00:00<?, ?it/s]

Checking rows:   0%|                                                               | 0/24200 [00:00<?, ?it/s]

C